In [151]:
import pandas as pd

In [152]:
def preprocess_common(data,test,n):
    data = data.drop(columns = ['date_recorded','gps_height','wpt_name','num_private','subvillage'
                                ,'lga','ward','recorded_by','extraction_type','management', 'management_group', 'payment' , 'quality_group', 'quantity' , 'source_type',  'waterpoint_type_group'])
    test = test.drop(columns = ['date_recorded','gps_height','wpt_name','num_private','subvillage'
                                ,'lga','ward','recorded_by','extraction_type','management', 'management_group', 'payment' , 'quality_group', 'quantity' , 'source_type',  'waterpoint_type_group'])

    data.permit = data.permit.astype(bool).astype(int)
    data.public_meeting = data.public_meeting.astype(bool).astype(int)
    test.permit = test.permit.astype(bool).astype(int)
    test.public_meeting = test.public_meeting.astype(bool).astype(int)
    
    def get_n(x,x_test,n):
    
        list_a = set(x.value_counts().index[:n].values) #Gets the top 10 values from the series
        x[~x.isin(list_a)] = 'Other' #vectorisation - boolean mask; tilda - inverting the boolean mask
        x_test[~x_test.isin(list_a)] = 'Other'
        return x,x_test
 
    data.funder,test.funder = get_n(data.funder,test.funder,n)
    data.installer,test.installer = get_n(data.installer,test.installer,n) 
    data.scheme_management,test.scheme_management = get_n(data.scheme_management,test.scheme_management,n) 
    
    def replace_with_median(data,test):
        region_median = data.groupby(['region_code'])[['latitude', 'longitude']].median()
        def geo_update(row, df_geo):
            row['longitude'] = df_geo.loc[row['region_code']]['longitude']
            row['latitude'] = df_geo.loc[row['region_code']]['latitude']
            return row

        data.loc[data['longitude']==0, ['longitude', 'latitude']] = \
            data[data['longitude']==0].apply(
                geo_update, df_geo=region_median, axis=1)[['longitude', 'latitude']]
        
        test.loc[test['longitude']==0, ['longitude', 'latitude']] = \
            test[test['longitude']==0].apply(
                geo_update, df_geo=region_median, axis=1)[['longitude', 'latitude']]
        return data,test
    
    data,test = replace_with_median(data,test)
    
    def replace_with_mode(data,test):
        df_scheme = data.groupby(['region'])['scheme_name'].agg(pd.Series.mode)

        def scheme_update(row, df_scheme):
            row['scheme_name'] = df_scheme[row['region']]
            return row

        data.loc[data['scheme_name'].isnull(), ['scheme_name']] = \
            data[data['scheme_name'].isnull()].apply(
                scheme_update, df_scheme=df_scheme, axis=1)[['scheme_name']]
        
        test.loc[test['scheme_name'].isnull(), ['scheme_name']] = \
            test[test['scheme_name'].isnull()].apply(
                scheme_update, df_scheme=df_scheme, axis=1)[['scheme_name']]
        return data,test
    
    data,test = replace_with_mode(data,test)
    data.scheme_name,test.scheme_name = get_n(data.scheme_name,test.scheme_name,n)
   
    data = data.drop(columns = ['region'])
    test = test.drop(columns = ['region'])
    return data,test




In [153]:
train_X = pd.read_csv(
    "data/train_feature.csv",
    index_col="id"
)

train_Y = pd.read_csv(
    "data/train_labels.csv",
    index_col="id"
)

test = pd.read_csv(
    "data/test.csv",
    index_col="id"
)

In [154]:
def oneHotEncode(data,test,cat_columns):
    data = pd.get_dummies(data,columns = cat_columns)
    test = pd.get_dummies(test,columns = cat_columns)
    return data, test

In [155]:
list(set(train_X.columns) - set(test.columns))

[]

In [156]:
P_train_X, P_test = preprocess_common(train_X,test,10)

<ipython-input-152-a22458b1298d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[~x.isin(list_a)] = 'Other' #vectorisation - boolean mask; tilda - inverting the boolean mask
<ipython-input-152-a22458b1298d>:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x_test[~x_test.isin(list_a)] = 'Other'
<ipython-input-152-a22458b1298d>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  x[~x.isin(list_a)] = 'Other' #vectorisation - boolean ma

In [157]:
list(set(train_X.columns) - set(test.columns))

[]

In [158]:
print(P_train_X.columns)
print(len(P_train_X.columns))

Index(['amount_tsh', 'funder', 'installer', 'longitude', 'latitude', 'basin',
       'region_code', 'district_code', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type_group', 'extraction_type_class', 'payment_type',
       'water_quality', 'quantity_group', 'source', 'source_class',
       'waterpoint_type'],
      dtype='object')
22


In [159]:
print(P_test.columns)
print(len(P_test.columns))

Index(['amount_tsh', 'funder', 'installer', 'longitude', 'latitude', 'basin',
       'region_code', 'district_code', 'population', 'public_meeting',
       'scheme_management', 'scheme_name', 'permit', 'construction_year',
       'extraction_type_group', 'extraction_type_class', 'payment_type',
       'water_quality', 'quantity_group', 'source', 'source_class',
       'waterpoint_type'],
      dtype='object')
22


In [160]:
cat_features=[i for i in P_train_X.columns if P_train_X.dtypes[i]=='object']
print(cat_features)
print(len(cat_features))

['funder', 'installer', 'basin', 'scheme_management', 'scheme_name', 'extraction_type_group', 'extraction_type_class', 'payment_type', 'water_quality', 'quantity_group', 'source', 'source_class', 'waterpoint_type']
13


In [161]:
cat_features=[i for i in P_test.columns if P_test.dtypes[i]=='object']
print(cat_features)
print(len(cat_features))

['funder', 'installer', 'basin', 'scheme_management', 'scheme_name', 'extraction_type_group', 'extraction_type_class', 'payment_type', 'water_quality', 'quantity_group', 'source', 'source_class', 'waterpoint_type']
13


In [162]:
# P_train_X, P_test = oneHotEncode(P_train_X, P_test,cat_features)

In [163]:
# P_train_X.to_csv("cleaned_1_train.csv",index=False)
# P_test.to_csv("cleaned_1_test.csv",index=False)

In [164]:
pd.options.display.max_columns = 40
P_train_X.describe(include='all')

,amount_tsh,funder,installer,longitude,latitude,basin,region_code,district_code,population,public_meeting,scheme_management,scheme_name,permit,construction_year,extraction_type_group,extraction_type_class,payment_type,water_quality,quantity_group,source,source_class,waterpoint_type
count,59400.000000,59400,59400,59400.000000,59400.000000,59400,59400.000000,59400.000000,59400.000000,59400.000000,59400,59400,59400.000000,59400.000000,59400,59400,59400,59400,59400,59400,59400,59400
unique,NaN,11,11,NaN,NaN,9,NaN,NaN,NaN,NaN,10,11,NaN,NaN,13,7,7,8,5,10,3,7
top,NaN,Other,Other,NaN,NaN,Lake Victoria,NaN,NaN,NaN,NaN,VWC,Other,NaN,NaN,gravity,gravity,never pay,soft,enough,spring,groundwater,communal standpipe
freq,NaN,36967,33013,NaN,NaN,10248,NaN,NaN,NaN,NaN,36793,33831,NaN,NaN,26780,26780,25348,50818,33186,17021,45794,28522
mean,317.650385,NaN,NaN,35.084525,-5.797934,NaN,15.297003,5.629747,179.909983,0.914899,NaN,NaN,0.705522,1300.652475,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
std,2997.574558,NaN,NaN,2.593514,2.811667,NaN,17.587406,9.633649,471.482176,0.279034,NaN,NaN,0.455812,951.620547,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.000000,NaN,NaN,29.607122,-11.649440,NaN,1.000000,0.000000,0.000000,0.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
25%,0.000000,NaN,NaN,33.091138,-8.541226,NaN,5.000000,2.000000,0.000000,1.000000,NaN,NaN,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
50%,0.000000,NaN,NaN,34.908743,-5.022018,NaN,12.000000,3.000000,25.000000,1.000000,NaN,NaN,1.000000,1986.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
75%,20.000000,NaN,NaN,37.178387,-3.342792,NaN,17.000000,5.000000,215.000000,1.000000,NaN,NaN,1.000000,2004.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [165]:
P_train_X, P_test = oneHotEncode(P_train_X, P_test,cat_features)

In [166]:
# P_train_X.to_csv("data/cleaned_1_train.csv")
# P_test.to_csv("data/cleaned_1_test.csv")

In [167]:
print(P_train_X.columns)
print(len(P_train_X.columns))

Index(['amount_tsh', 'longitude', 'latitude', 'region_code', 'district_code',
       'population', 'public_meeting', 'permit', 'construction_year',
       'funder_Danida',
       ...
       'source_class_groundwater', 'source_class_surface',
       'source_class_unknown', 'waterpoint_type_cattle trough',
       'waterpoint_type_communal standpipe',
       'waterpoint_type_communal standpipe multiple', 'waterpoint_type_dam',
       'waterpoint_type_hand pump', 'waterpoint_type_improved spring',
       'waterpoint_type_other'],
      dtype='object', length=121)
121


In [168]:
print(P_test.columns)
print(len(P_test.columns))

Index(['amount_tsh', 'longitude', 'latitude', 'region_code', 'district_code',
       'population', 'public_meeting', 'permit', 'construction_year',
       'funder_Danida',
       ...
       'source_class_groundwater', 'source_class_surface',
       'source_class_unknown', 'waterpoint_type_cattle trough',
       'waterpoint_type_communal standpipe',
       'waterpoint_type_communal standpipe multiple', 'waterpoint_type_dam',
       'waterpoint_type_hand pump', 'waterpoint_type_improved spring',
       'waterpoint_type_other'],
      dtype='object', length=121)
121


In [169]:
list(set(P_train_X.columns) - set(P_test.columns))

[]

In [171]:
P_train_X.to_csv("data/cleaned_1_train.csv")
P_test.to_csv("data/cleaned_1_test.csv")